# Semantic Kernel با یکپارچه‌سازی سرور OpenBnB MCP

این دفترچه نشان می‌دهد چگونه از Semantic Kernel با سرور واقعی OpenBnB MCP برای جستجوی اقامتگاه‌های واقعی Airbnb با استفاده از MCPStdioPlugin استفاده کنید. برای دسترسی به LLM، از Microsoft Foundry استفاده می‌کند. برای تنظیم متغیرهای محیطی خود، می‌توانید درس [تنظیم](/00-course-setup/README.md) را دنبال کنید.


## وارد کردن بسته‌های مورد نیاز


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## ایجاد اتصال افزونه MCP

ما با استفاده از MCPStdioPlugin به [سرور OpenBnB MCP](https://github.com/openbnb-org/mcp-server-airbnb) متصل خواهیم شد. این سرور قابلیت جستجوی Airbnb را از طریق بسته @openbnb/mcp-server-airbnb فراهم می‌کند.


## ایجاد کلاینت

در این نمونه، از Microsoft Foundry برای دسترسی به LLM خود استفاده خواهیم کرد. مطمئن شوید متغیرهای محیطی شما به درستی تنظیم شده‌اند.


## پیکربندی محیط

تنظیمات Azure OpenAI را پیکربندی کنید. مطمئن شوید که متغیرهای محیطی زیر تنظیم شده‌اند:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## درک یکپارچه‌سازی OpenBnB MCP

این دفترچه به **سرور واقعی OpenBnB MCP** متصل می‌شود که عملکرد جستجوی واقعی Airbnb را فراهم می‌کند.

### نحوه کار:

1. **MCPStdioPlugin**: از ارتباط ورودی/خروجی استاندارد با سرور MCP استفاده می‌کند
2. **بسته واقعی NPM**: از طریق npx بسته `@openbnb/mcp-server-airbnb` را دانلود و اجرا می‌کند
3. **داده زنده**: داده واقعی ملک‌های Airbnb را از APIهای آن‌ها بازمی‌گرداند
4. **کشف عملکردها**: عامل به‌صورت خودکار عملکردهای موجود از سرور MCP را کشف می‌کند

### عملکردهای موجود:

سرور OpenBnB MCP معمولاً موارد زیر را ارائه می‌دهد:
- **search_listings** - جستجوی ملک‌های Airbnb بر اساس مکان و معیارها
- **get_listing_details** - دریافت اطلاعات دقیق درباره ملک‌های مشخص
- **check_availability** - بررسی در دسترس بودن برای تاریخ‌های مشخص
- **get_reviews** - دریافت نظرات مربوط به ملک‌ها
- **get_host_info** - دریافت اطلاعات درباره میزبان‌های ملک

### پیش‌نیازها:

- نصب **Node.js** بر روی سیستم شما
- اتصال **اینترنت** برای دانلود بسته سرور MCP
- در دسترس بودن **NPX** (همراه با Node.js ارائه می‌شود)

### آزمایش اتصال:

می‌توانید به‌صورت دستی با اجرای دستور زیر سرور MCP را آزمایش کنید:
```bash
npx -y @openbnb/mcp-server-airbnb
```

این دستور سرور OpenBnB MCP را دانلود و راه‌اندازی می‌کند که Semantic Kernel سپس برای دریافت داده واقعی Airbnb به آن متصل می‌شود.


## اجرای عامل با سرور OpenBnB MCP

اکنون عامل هوش مصنوعی را اجرا می‌کنیم که به سرور OpenBnB MCP متصل می‌شود تا به جستجوی اقامتگاه‌های واقعی Airbnb در استکهلم برای ۲ بزرگسال و ۱ کودک بپردازد. در صورت تمایل می‌توانید فهرست `user_inputs` را برای تغییر معیارهای جستجو اصلاح کنید.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# خلاصه
تبریک می‌گوییم! شما با موفقیت یک عامل هوش مصنوعی ساخته‌اید که با جستجوی اقامت در دنیای واقعی با استفاده از پروتکل مدل زمینه (MCP) یکپارچه شده است:

## فناوری‌های استفاده شده:
- Semantic Kernel - برای ساخت عامل‌های هوشمند با Azure OpenAI
- Microsoft Foundry - برای قابلیت‌های LLM و تکمیل چت
- MCP (پروتکل مدل زمینه) - برای یکپارچه‌سازی استاندارد ابزارها
- OpenBnB MCP Server - برای عملکرد واقعی جستجوی Airbnb
- Node.js/NPX - برای اجرای سرور خارجی MCP

## آنچه یاد گرفته‌اید:
- ادغام MCP: اتصال عامل‌های Semantic Kernel به سرورهای خارجی MCP
- دسترسی به داده‌های زنده: جستجوی واقعی املاک Airbnb از طریق APIهای زنده
- ارتباط پروتکل: استفاده از ارتباط stdio بین عامل و سرور MCP
- کشف تابع: کشف خودکار توابع موجود از سرورهای MCP
- پاسخ‌های جریانی: ثبت و ضبط تماس‌های تابع در زمان واقعی
- رندر HTML: فرمت‌بندی پاسخ‌های عامل با جداول استایل‌داده شده و نمایش‌های تعاملی

## مراحل بعدی:
- ادغام سرورهای MCP اضافی (هواشناسی، پروازها، رستوران‌ها)
- ساخت سیستم چندعامله ترکیب شده با پروتکل‌های MCP و A2A
- ایجاد سرورهای MCP سفارشی برای منابع داده خودتان
- پیاده‌سازی حافظه گفتگوی پایدار در جلسات مختلف
- استقرار عامل در Azure Functions با ارکستراسیون سرور MCP
- افزودن احراز هویت کاربر و قابلیت‌های رزرو


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما در تلاش برای دقت هستیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است دارای خطاها یا نادرستی‌هایی باشند. سند اصلی به زبان بومی آن باید به‌عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حیاتی، استفاده از ترجمه حرفه‌ای انسانی توصیه می‌شود. ما مسئول هیچ گونه سوء تفاهم یا تفسیر نادرست ناشی از استفاده از این ترجمه نیستیم.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
